# 인공신경망 4조

20202995 장진영  
20211522 박상은  
20192562 강성욱  
20192596 백현우

## basic setting

## dataset load (by merge)

In [ ]:
# 데이터 불러오기
testing_set_merge = pd.read_csv('emnist-bymerge-test.csv')
training_set_merge = pd.read_csv('emnist-bymerge-train.csv')

# 훈련 데이터와 레이블 추출
train_y_merge = np.array(training_set_merge.iloc[:, 0].values)
train_x_merge = np.array(training_set_merge.iloc[:, 1:].values)

# 테스트 데이터와 레이블 추출
test_y_merge = np.array(testing_set_merge.iloc[:, 0].values)
test_x_merge = np.array(testing_set_merge.iloc[:, 1:].values)

# 데이터 정규화
train_x_merge = train_x_merge / 255.0
test_x_merge = test_x_merge / 255.0

# 이미지 크기 재조정 (데이터 차원 변경)
images_height = 28
images_width = 28
train_x_merge = train_x_merge.reshape(train_x_merge.shape[0], images_height, images_width, 1)
test_x_merge = test_x_merge.reshape(test_x_merge.shape[0], images_height, images_width, 1)

# 클래스 수 = 47
number_of_classes = 47

# 레이블 원-핫 인코딩
train_y_merge = tf.keras.utils.to_categorical(train_y_merge, number_of_classes)
test_y_merge = tf.keras.utils.to_categorical(test_y_merge, number_of_classes)

VALIDATION_SPLIT = 0.15 
train_x_merge, val_x_merge, train_y_merge, val_y_merge = train_test_split(train_x_merge, train_y_merge, 
                                                                          test_size=VALIDATION_SPLIT, random_state=42)

## dataset load (balanced)

In [ ]:
# 데이터 불러오기
testing_set_balance = pd.read_csv('emnist-balanced-test.csv')
training_set_balance = pd.read_csv('emnist-balanced-train.csv')


# 훈련 데이터와 레이블 추출
train_y_balance = np.array(training_set_balance.iloc[:, 0].values)
train_x_balance = np.array(training_set_balance.iloc[:, 1:].values)

# 테스트 데이터와 레이블 추출
test_y_balance = np.array(testing_set_balance.iloc[:, 0].values)
test_x_balance = np.array(testing_set_balance.iloc[:, 1:].values)

# 데이터 정규화
train_x_balance = train_x_balance / 255.0
test_x_balance = test_x_balance / 255.0

# 이미지 크기 재조정 (데이터 차원 변경)
images_height = 28
images_width = 28
train_x_balance = train_x_balance.reshape(train_x_balance.shape[0], images_height, images_width, 1)
test_x_balance = test_x_balance.reshape(test_x_balance.shape[0], images_height, images_width, 1)

# 클래스 수 = 47
number_of_classes = 47

# 레이블 원-핫 인코딩
train_y_balance = tf.keras.utils.to_categorical(train_y_balance, number_of_classes)
test_y_balance = tf.keras.utils.to_categorical(test_y_balance, number_of_classes)

VALIDATION_SPLIT = 0.15
train_x_balance, val_x_balance, train_y_balance, val_y_balance = train_test_split(train_x_balance, train_y_balance, 
                                                                              test_size=VALIDATION_SPLIT, random_state=42)

## LeNet (by_merge)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, Input,AveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Model

### hyperparameter 변경하며 최적화 진행

In [ ]:
# Hyper parameter setting
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.001
PATIENCE = 10
MIN_LEARNING_RATE = 0.0001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'tanh'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']

CHECK_POINT_MERGE = 'Best_points_LeNet5_merge.h5'
CHECK_POINT_BALANCE = 'Best_points_LeNet5_balance.h5'
CHECK_POINT_BALANCE_AUG = 'Best_points_LeNet5_balance_aug.h5'

In [ ]:
model_merge = tf.keras.Sequential([
    Conv2D(6, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, input_shape=(28, 28, 1), padding='same'),
    AveragePooling2D(),
    Conv2D(16, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION),
    AveragePooling2D(),
    Flatten(),
    Dense(120, activation=DENSE_LAYER_ACTIVATION),
    Dense(84, activation=DENSE_LAYER_ACTIVATION),
    Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)
])

model_merge.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=LOSS_FUNCTION,
              metrics=METRICS)


callbacks_merge = [
    ModelCheckpoint(CHECK_POINT_MERGE, verbose=1, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=PATIENCE, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=PATIENCE, factor=DECAY_FACTOR, min_lr=MIN_LEARNING_RATE)
]


In [ ]:
start_time = time.time()  # 학습 시작 시간

# 모델 학습
history_merge = model_merge.fit(train_x_merge, train_y_merge, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_merge, val_y_merge), callbacks=callbacks_merge)

training_time = time.time() - start_time

In [ ]:
# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = model_merge.predict(test_x_merge)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_merge)

# 테스트 데이터에 대한 예측 결과 처리
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_merge, axis=1)

# 정확도 계산
correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

# 결과 출력
print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.") 

### hyperparameter 별 결과 (accuracy / training time / inference time)

#### batch_size = 32   &   learning_rate = 0.001

Correct Predictions: 103526  
Total Predictions: 116322  
정확도: 0.89  
훈련시간 : 1292.55 seconds  
평균 추론 시간: 0.00003 seconds  

------------

#### batch_size = 32   &   learning_rate = 0.01

Correct Predictions: 84915  
Total Predictions: 116322  
정확도: 0.73  
훈련시간 : 1157.37 seconds  
평균 추론 시간: 0.00002 seconds  

------------

#### batch_size = 64   &   learning_rate = 0.001

Correct Predictions: 104177  
Total Predictions: 116322  
정확도: 0.90  
훈련시간 : 1029.96 seconds.  
평균 추론 시간: 0.00003 seconds.  

---------

#### batch_size = 64   &   learning_rate = 0.01
Correct Predictions: 90886  
Total Predictions: 116322  
정확도: 0.78  
훈련시간 : 451.53 seconds.  
평균 추론 시간: 0.00002 seconds.  

----------------
---------------
---------------

## LeNet (balanced)

In [ ]:
start_time = time.time()  # 학습 시작 시간

# 모델 학습
history_balance = model_balance.fit(train_x_balance, train_y_balance, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_balance, val_y_balance), callbacks=callbacks_balance)

training_time = time.time() - start_time

In [ ]:
# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = model_balance.predict(test_x_balance)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

# 테스트 데이터에 대한 예측 결과 처리
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

# 정확도 계산
correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

# 결과 출력
print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.5f} seconds.") #각 샘플에 대한 추론 시간을 개별적으로 측정함으로써, 추론에 필요한 시간을 더 정확히 파악

### hyperparameter 별 결과 (accuracy / training time / inference time)

#### batch_size = 32   &   learning_rate = 0.001

Correct Predictions: 16355  
Total Predictions: 18799  
정확도: 0.87  
훈련시간 : 106.57 seconds  
평균 추론 시간: 0.00003 seconds  

------------

#### batch_size = 32   &   learning_rate = 0.01

Correct Predictions: 13347  
Total Predictions: 18799  
정확도: 0.71  
훈련시간 : 184.75 seconds  
평균 추론 시간: 0.00002 seconds  

------------

#### batch_size = 64   &   learning_rate = 0.001

Correct Predictions: 16319  
Total Predictions: 18799  
정확도: 0.87  
훈련시간 : 168.64 seconds.  
평균 추론 시간: 0.00003 seconds.  
 

---------

#### batch_size = 64   &   learning_rate = 0.01
Correct Predictions: 14158  
Total Predictions: 18799  
정확도: 0.75  
훈련시간 : 141.02 seconds.  
평균 추론 시간: 0.00002 seconds.  

--------
----------
---------

## ResNet (by_merge)

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights=None, include_top=False, input_tensor=Input(shape=(28, 28, 1)))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(120, activation=DENSE_LAYER_ACTIVATION)(x)
x = Dense(84, activation=DENSE_LAYER_ACTIVATION)(x)
output = Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)(x)

# 새로운 모델 정의
model_merge = Model(inputs=base_model.input, outputs=output)


model_merge.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=LOSS_FUNCTION,
              metrics=METRICS)


callbacks_merge = [
    ModelCheckpoint(CHECK_POINT_MERGE, verbose=1, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=PATIENCE, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=PATIENCE, factor=DECAY_FACTOR, min_lr=MIN_LEARNING_RATE)
]


In [ ]:
start_time = time.time()  # 학습 시작 시간

# 모델 학습
history_merge = model_merge.fit(train_x_merge, train_y_merge, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_merge, val_y_merge), callbacks=callbacks_merge)

training_time = time.time() - start_time

In [ ]:
# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = model_merge.predict(test_x_merge)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_merge)

# 테스트 데이터에 대한 예측 결과 처리
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_merge, axis=1)

# 정확도 계산
correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

# 결과 출력
print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.") #각 샘플에 대한 추론 시간을 개별적으로 측정함으로써, 추론에 필요한 시간을 더 정확히 파악

### hyperparameter 별 결과 (accuracy / training time / inference time)

#### batch_size = 64   &   learning_rate = 0.001

Correct Predictions: 105996  
Total Predictions: 116322  
정확도: 0.91  
훈련시간 : 7323.24 seconds  
평균 추론 시간: 0.0003 seconds  

------------

#### batch_size = 64   &   learning_rate = 0.01

Correct Predictions: 105720  
Total Predictions: 116322  
정확도: 0.91  
훈련시간 : 13358.46 seconds  
평균 추론 시간: 0.0001 seconds  

------------

#### batch_size = 128   &   learning_rate = 0.001

Correct Predictions: 105642  
Total Predictions: 116322  
정확도: 0.91  
훈련시간 : 4413.92 seconds.  
평균 추론 시간: 0.0003 seconds.  

---------

#### batch_size = 128   &   learning_rate = 0.01
Correct Predictions: 105075  
Total Predictions: 116322  
정확도: 0.90  
훈련시간 : 2858.69 seconds.  
평균 추론 시간: 0.0002 seconds.  

-----------
----
----

## ResNet(balanced)

In [ ]:
start_time = time.time()  # 학습 시작 시간

# 모델 학습
history_balance = model_balance.fit(train_x_balance, train_y_balance, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_balance, val_y_balance), callbacks=callbacks_balance)

training_time = time.time() - start_time

In [ ]:
# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = model_balance.predict(test_x_balance)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

# 테스트 데이터에 대한 예측 결과 처리
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

# 정확도 계산
correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

# 결과 출력
print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.") #각 샘플에 대한 추론 시간을 개별적으로 측정함으로써, 추론에 필요한 시간을 더 정확히 파악

### hyperparameter 별 결과 (accuracy / training time / inference time)

#### batch_size = 64   &   learning_rate = 0.001

Correct Predictions: 16919  
Total Predictions: 18799  
정확도: 0.90  
훈련시간 : 1150.34 seconds  
평균 추론 시간: 0.0002 seconds  

------------

#### batch_size = 64   &   learning_rate = 0.01


Correct Predictions: 16167  
Total Predictions: 18799  
정확도: 0.86  
훈련시간 : 776.97 seconds  
평균 추론 시간: 0.0012 seconds  

------------

#### batch_size = 128   &   learning_rate = 0.001

Correct Predictions: 16845  
Total Predictions: 18799  
정확도: 0.90  
훈련시간 : 1040.00 seconds.  
평균 추론 시간: 0.0003 seconds.  

---------

#### batch_size = 128   &   learning_rate = 0.01
Correct Predictions: 16645  
Total Predictions: 18799  
정확도: 0.89  
훈련시간 : 1650.16 seconds.  
평균 추론 시간: 0.0002 seconds.  

-----------
----
----

## Data augmentation 

|augmentation|rotation_range|width_shift_range|height_shift_range|
|:--:|:--:|:--:|:--:|
|augmentation 1|20|0.2|0.2|
|augmentation 2|30|0.1|0.1|
|augmentation 3|45|0.05|0.05|


In [ ]:
# 데이터 스케일링 (StandardScaler 사용)
scaler = StandardScaler()

img_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
)

img_generator2 = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

img_generator3 = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.05,
    height_shift_range=0.05,
)

aug1_train_data = img_generator.flow(train_x_balance,train_y_balance, batch_size=len(train_x_balance))
aug2_train_data = img_generator2.flow(train_x_balance,train_y_balance, batch_size=len(train_x_balance))
aug3_train_data = img_generator3.flow(train_x_balance,train_y_balance, batch_size=len(train_x_balance))

aug1_x,aug1_y = aug1_train_data.next()
aug2_x,aug2_y = aug2_train_data.next()
aug3_x,aug3_y = aug3_train_data.next()

x_combined = np.concatenate((train_x_balance,aug1_x,aug2_x,aug3_x))
y_combined = np.concatenate((train_y_balance,aug1_y,aug2_y,aug3_y))

shuffle_index = np.random.permutation(len(x_combined))
x_combined_shff = x_combined[shuffle_index]
y_combined_shff = y_combined[shuffle_index]


aug1_valid_data = img_generator.flow(val_x_balance,val_y_balance, batch_size=len(val_x_balance))
aug2_valid_data = img_generator2.flow(val_x_balance,val_y_balance, batch_size=len(val_x_balance))
aug3_valid_data = img_generator3.flow(val_x_balance,val_y_balance, batch_size=len(val_x_balance))

aug1_val_x,aug1_val_y = aug1_valid_data.next()
aug2_val_x,aug2_val_y = aug2_valid_data.next()
aug3_val_x,aug3_val_y = aug3_valid_data.next()

x_val_combined = np.concatenate((val_x_balance,aug1_val_x,aug2_val_x,aug3_val_x))
y_val_combined = np.concatenate((val_y_balance,aug1_val_y,aug2_val_y,aug3_val_y))

shuffle_index2 = np.random.permutation(len(x_val_combined))
x_combined_val_shff = x_val_combined[shuffle_index2]
y_combined_val_shff = y_val_combined[shuffle_index2]

#이미지를 2D 배열로 변환
n_samples, height, width, n_channels = x_combined_shff.shape
x_combined_shff = x_combined_shff.reshape((n_samples, -1))

# 학습 데이터 스케일러 학습 및 변환
scaler.fit(x_combined_shff)
x_combined_shff = scaler.transform(x_combined_shff)

# 스케일된 데이터를 다시 원래 이미지 형태로 변환
x_combined_shff = x_combined_shff.reshape((n_samples, height, width, n_channels))

# 검증 데이터 동일한 스케일러로 변환
n_val_samples = x_combined_val_shff.shape[0]
x_combined_val_shff = x_combined_val_shff.reshape((n_val_samples, -1))
x_combined_val_shff = scaler.transform(x_combined_val_shff)
x_combined_val_shff = x_combined_val_shff.reshape((n_val_samples, height, width, n_channels))

# 테스트 데이터 동일한 스케일로 변환
n_val_samples = test_x_balance.shape[0]
test_x_balance = test_x_balance.reshape((n_val_samples, -1))
test_x_balance = scaler.transform(test_x_balance)
test_x_balance = test_x_balance.reshape((n_val_samples, height, width, n_channels))

### 변수명 변경
train_x_balance  →  x_combined_shff  
train_y_balance  →  y_combined_shff  
val_x_balance  →  x_combined_val_shff  
val_y_balance  →  y_combined_val_shff

### LeNet

In [ ]:
model_balance_aug = tf.keras.Sequential([
    Conv2D(6, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, input_shape=(28, 28, 1), padding='same'),
    AveragePooling2D(),
    Conv2D(16, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION),
    AveragePooling2D(),
    Flatten(),
    Dense(120, activation=DENSE_LAYER_ACTIVATION),
    Dense(84, activation=DENSE_LAYER_ACTIVATION),
    Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)
])


model_balance_aug.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=LOSS_FUNCTION,
              metrics=METRICS)


callbacks_balance = [
    ModelCheckpoint(CHECK_POINT_BALANCE_AUG, verbose=1, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=PATIENCE, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=PATIENCE, factor=DECAY_FACTOR, min_lr=MIN_LEARNING_RATE)
]


In [ ]:
start_time = time.time()  # 학습 시작 시간

# 모델 학습
history_balance_aug = model_balance_aug.fit(x_combined_shff, y_combined_shff, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(x_combined_val_shff, y_combined_val_shff), callbacks=callbacks_balance)

training_time = time.time() - start_time

In [ ]:
# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = model_balance_aug.predict(test_x_balance)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

# 테스트 데이터에 대한 예측 결과 처리
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

# 정확도 계산
correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

# 결과 출력
print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.5f} seconds.")

Correct Predictions: 16425  
Total Predictions: 18799  
정확도: 0.87  
훈련시간 : 2090.59 seconds.  
평균 추론 시간: 0.00003 seconds.  

-----------

### ResNet

In [ ]:
base_model = ResNet50(weights=None, include_top=False, input_tensor=Input(shape=(28, 28, 1)))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(120, activation=DENSE_LAYER_ACTIVATION)(x)
x = Dense(84, activation=DENSE_LAYER_ACTIVATION)(x)
output = Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)(x)

# 새로운 모델 정의
model_balance_aug = Model(inputs=base_model.input, outputs=output)

model_balance_aug.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=LOSS_FUNCTION,
              metrics=METRICS)


callbacks_balance = [
    ModelCheckpoint(CHECK_POINT_BALANCE, verbose=1, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=PATIENCE, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=PATIENCE, factor=DECAY_FACTOR, min_lr=MIN_LEARNING_RATE)
]


In [ ]:
start_time = time.time()  # 학습 시작 시간

# 모델 학습
history_balance_aug = model_balance_aug.fit(x_combined_shff, y_combined_shff, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(x_combined_val_shff, y_combined_val_shff), callbacks=callbacks_balance)

training_time = time.time() - start_time

In [ ]:
# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = model_balance_aug.predict(test_x_balance)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

# 테스트 데이터에 대한 예측 결과 처리
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

# 정확도 계산
correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

# 결과 출력
print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")

Correct Predictions: 16882  
Total Predictions: 18799  
정확도: 0.90  
훈련시간 : 2791.82 seconds.  
평균 추론 시간: 0.0003 seconds.  

---
---
---

## Pre-trained model training

### ResNet-34

In [ ]:
from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

# Residual Unit 정의
class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.strides = strides
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            Activation(self.activation),
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'filters': self.filters,
            'strides': self.strides,
            'activation': tf.keras.activations.serialize(self.activation),
        })
        return config


In [ ]:
# Define ResNet-34
Resnet34 = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=3, strides=1, input_shape=[28, 28, 1]), #FEATURE SIZE 유지
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(DENSE_LAYER_ACTIVATION),
])
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    Resnet34.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

Resnet34.add(tf.keras.layers.GlobalAvgPool2D())
Resnet34.add(tf.keras.layers.Flatten())
Resnet34.add(tf.keras.layers.Dense(47, activation=FINAL_LAYER_ACTIVATION))

In [ ]:
Resnet34.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=LOSS_FUNCTION,
              metrics=METRICS)


start_time = time.time() 

# 모델 학습
history_balance = Resnet34.fit(train_x_balance, train_y_balance, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_balance, val_y_balance), callbacks=callbacks_balance)

training_time = time.time() - start_time

# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = Resnet34.predict(test_x_balance)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")

Correct Predictions: 16820  
Total Predictions: 18799  
정확도: 0.89  
훈련시간 : 1067.71 seconds.  
평균 추론 시간: 0.0002 seconds.  

### DenseNet

In [ ]:
# Define a simple DenseNet model
def dense_block(x, blocks, name):
    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x

def conv_block(x, growth_rate, name):
    x1 = BatchNormalization(axis=3, epsilon=1.001e-5, name=name + '_0_bn')(x)
    x1 = Activation('relu', name=name + '_0_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(x1)
    x1 = BatchNormalization(axis=3, epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = Activation('relu', name=name + '_1_relu')(x1)
    x1 = Conv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(x1)
    x = Concatenate(axis=3, name=name + '_concat')([x, x1])
    return x

def transition_block(x, reduction, name):
    x = BatchNormalization(axis=3, epsilon=1.001e-5, name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[3] * reduction), 1, use_bias=False, name=name + '_conv')(x)
    x = AveragePooling2D(1, strides=1, name=name + '_pool')(x)  # AveragePooling2D의 풀링 크기를 (1, 1)로 변경
    return x

def DenseNet(input_shape, classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(inputs)
    x = BatchNormalization(axis=3, epsilon=1.001e-5, name='conv1/bn')(x)
    x = Activation('relu', name='conv1/relu')(x)
    x = AveragePooling2D(3, strides=2, name='pool1')(x)

    x = dense_block(x, 6, name='conv2')
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, 12, name='conv3')
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, 24, name='conv4')
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, 16, name='conv5')

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    outputs = Dense(classes, activation='softmax', name='fc')(x)

    model = Model(inputs, outputs, name='densenet')

    return model

# Define model
model = DenseNet((28, 28, 1), number_of_classes)


Correct Predictions: 16741  
Total Predictions: 18799  
정확도: 0.89  
훈련시간 : 3352.60 seconds.  
평균 추론 시간: 0.0005 seconds.  

### EffcientNet

In [ ]:
# 입력 레이어 정의
input_tensor = Input(shape=(28, 28, 1))

# EfficientNetB0 모델 불러오기 (최상위 레이어 제외)
base_model = EfficientNetB0(weights=None, include_top=False, input_tensor=input_tensor)

# GlobalAveragePooling2D 레이어 추가
x = GlobalAveragePooling2D()(base_model.output)

# 분류 레이어 추가 (47개 클래스)
output_tensor = Dense(47, activation='softmax')(x)

# 최종 모델 정의
model = Model(inputs=input_tensor, outputs=output_tensor)

Correct Predictions: 16737  
Total Predictions: 18799  
정확도: 0.8903  
훈련시간 : 5942.75 seconds.  
평균 추론 시간: 0.0004 seconds.  

### MobileNet  

In [ ]:
## 모델 최소 입력 크기를 맞추기 위한 padding 정의
import numpy as np

# Zero-padding 함수 정의
def zero_pad_images(images, pad_height, pad_width):
    padded_images = np.pad(images, ((0, 0), (pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='constant', constant_values=0)
    return padded_images

# Zero-padding 적용
pad_height = 2  # 수직 방향으로 2 픽셀 패딩 추가
pad_width = 2   # 수평 방향으로 2 픽셀 패딩 추가

train_x_balanced = zero_pad_images(train_x_balanced, pad_height, pad_width)
val_x_balanced = zero_pad_images(val_x_balanced, pad_height, pad_width)
test_x_balanced = zero_pad_images(test_x_balanced, pad_height, pad_width)

In [ ]:
base_model = tf.keras.applications.MobileNet(weights=None, include_top=False, input_shape=(32, 32, 1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(47, activation=FINAL_LAYER_ACTIVATION)(x)

model = Model(inputs=base_model.input, outputs=output)

Correct Predictions: 16647  
Total Predictions: 18799  
정확도: 0.8855  
훈련시간 : 1099.04 seconds.  
평균 추론 시간: 0.0008 seconds.  

---

#### pre-trained이용의 의의를 위해 (28,28,1) → (224,224,3) 변환 시도

In [ ]:
import tensorflow as tf

# Resizing 레이어 추가
resizing_layer = tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True)

# MobileNet 모델 로드 (weights는 ImageNet의 가중치 사용, include_top은 fully connected layers 포함 여부)
mobilenet_model = tf.keras.applications.MobileNet(weights="imagenet", include_top=False)

# Global Average Pooling 레이어 추가
global_average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()

# Classification을 위한 Dense 레이어 추가
dense_layer = tf.keras.layers.Dense(units=47, activation='softmax')  # 분류 클래스 수에 맞게 units 설정

# 모델 구성
model = tf.keras.Sequential([
    resizing_layer,
    mobilenet_model,
    global_average_pooling_layer,
    dense_layer
])

# 모델 빌드
model.build((None, 224, 224, 3))

# 모델 요약
model.summary()


In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=LOSS_FUNCTION,
              metrics=METRICS)


#### compile함수를 지원하지 않는 error 발생

In [ ]:
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
C:\Users\User\AppData\Local\Temp\ipykernel_20168\374034419.py in <module>
----> 1 model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
      2               loss=LOSS_FUNCTION,
      3               metrics=METRICS)

AttributeError: 'Resizing' object has no attribute 'compile'

----
---
---

## Model Design

### Model  1

In [ ]:
def Inception_residual_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4):
    # 1st path: 1x1 Convolution
    path1 = Conv2D(filters=f1, kernel_size=(1, 1), padding='same', activation='relu')(input_layer)

    # 2nd path: 1x1 Convolution followed by 3x3 Convolution
    path2 = Conv2D(filters=f2_conv1, kernel_size=(1, 1), padding='same', activation='relu')(input_layer)
    path2 = Conv2D(filters=f2_conv3, kernel_size=(3, 3), padding='same', activation='relu')(path2)

    # 3rd path: 1x1 Convolution followed by 5x5 Convolution
    path3 = Conv2D(filters=f3_conv1, kernel_size=(1, 1), padding='same', activation='relu')(input_layer)
    path3 = Conv2D(filters=f3_conv5, kernel_size=(5, 5), padding='same', activation='relu')(path3)

    # 4th path: Max Pooling followed by 1x1 Convolution
    path4 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(input_layer)
    path4 = Conv2D(filters=f4, kernel_size=(1, 1), padding='same', activation='relu')(path4)

    # Concatenate all paths
    output_layer = concatenate([path1, path2, path3, path4], axis=-1)

    # Residual connection: match dimensions of input and output if necessary
    if input_layer.shape[-1] != output_layer.shape[-1]:
        input_layer = Conv2D(filters=output_layer.shape[-1], kernel_size=(1, 1), padding='same')(input_layer)

    # Add the input layer to the output layer
    output_layer = add([input_layer, output_layer])
    output_layer = Activation('relu')(output_layer)

    return output_layer

# Example of usage
input_layer = tf.keras.layers.Input(shape=(28, 28, 1))
X = Conv2D(filters=64, kernel_size=(5, 5), strides=2, padding='valid', activation='relu')(input_layer)
X = MaxPooling2D(pool_size=(2,2), strides=2)(X)
X = Inception_residual_block(X, f1=64, f2_conv1=96, f2_conv3=128, f3_conv1=16, f3_conv5=32, f4=32)
X = Inception_residual_block(X, f1=160, f2_conv1=112, f2_conv3=224, f3_conv1=24, f3_conv5=64, f4=64)
X = Inception_residual_block(X, f1=128, f2_conv1=128, f2_conv3=256, f3_conv1=24, f3_conv5=64, f4=64)
X = Inception_residual_block(X, f1=112, f2_conv1=144, f2_conv3=288, f3_conv1=32, f3_conv5=64, f4=64)
# Global Average pooling layer
X = GlobalAveragePooling2D(name='GAPL')(X)
# Dropout layer
X = Dropout(0.4)(X)
# Output layer
output_layer = Dense(47, activation='softmax')(X)
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=LOSS_FUNCTION,
              metrics=METRICS)


start_time = time.time() 

# 모델 학습
history_balance = model.fit(train_x_balance, train_y_balance, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_balance, val_y_balance), callbacks=callbacks_balance)

training_time = time.time() - start_time

# Evaluate model
start_time = time.time()  # 추론 시작 시간
predictions = model.predict(test_x_balance)
end_time = time.time()  # 추론 종료 시간
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")

Correct Predictions: 16861  
Total Predictions: 18799  
정확도: 0.90  
훈련시간 : 1674.95 seconds.  
평균 추론 시간: 0.0002 seconds.  

### Model 2

In [ ]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4):
    # 1st path: 1x1 Convolution
    path1 = Conv2D(filters=f1, kernel_size=(1, 1), padding='same', activation='relu')(input_layer)

    # 2nd path: 1x1 Convolution followed by 3x3 Convolution
    path2 = Conv2D(filters=f2_conv1, kernel_size=(1, 1), padding='same', activation='relu')(input_layer)
    path2 = Conv2D(filters=f2_conv3, kernel_size=(3, 3), padding='same', activation='relu')(path2)

    # 3rd path: 1x1 Convolution followed by 5x5 Convolution
    path3 = Conv2D(filters=f3_conv1, kernel_size=(1, 1), padding='same', activation='relu')(input_layer)
    path3 = Conv2D(filters=f3_conv5, kernel_size=(5, 5), padding='same', activation='relu')(path3)

    # 4th path: Max Pooling followed by 1x1 Convolution
    path4 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(input_layer)
    path4 = Conv2D(filters=f4, kernel_size=(1, 1), padding='same', activation='relu')(path4)

    # Concatenate all paths
    output_layer = concatenate([path1, path2, path3, path4], axis=-1)

    return output_layer

# Example of usage
input_layer = tf.keras.layers.Input(shape=(28, 28, 1))
X = Conv2D(filters=64, kernel_size=(5, 5), strides=2, padding='valid', activation='relu')(input_layer)
X = MaxPooling2D(pool_size=(2,2), strides=2)(X)
X = Inception_residual_block(X, f1=64, f2_conv1=96, f2_conv3=128, f3_conv1=16, f3_conv5=32, f4=32)
X = Inception_residual_block(X, f1=160, f2_conv1=112, f2_conv3=224, f3_conv1=24, f3_conv5=64, f4=64)
X = Inception_residual_block(X, f1=128, f2_conv1=128, f2_conv3=256, f3_conv1=24, f3_conv5=64, f4=64)
X = Inception_residual_block(X, f1=112, f2_conv1=144, f2_conv3=288, f3_conv1=32, f3_conv5=64, f4=64)
# Global Average pooling layer
X = GlobalAveragePooling2D(name='GAPL')(X)
# Dropout layer
X = Dropout(0.4)(X)
# Output layer
output_layer = Dense(47, activation='softmax')(X)
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

Correct Predictions: 16784  
Total Predictions: 18799  
정확도: 0.89  
훈련시간 : 1300.10 seconds.  
평균 추론 시간: 0.0002 seconds.  

### Model 3

In [ ]:
model = tf.keras.Sequential([
    Conv2D(16, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, input_shape=(28, 28, 1), padding='same'),
    AveragePooling2D(),
    Conv2D(32, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(64, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(128, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(128, activation=DENSE_LAYER_ACTIVATION),
    Dropout(0.5),
    Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)
])

Correct Predictions: 16773  
Total Predictions: 18799  
정확도: 0.8922  
훈련시간 : 360.5077 seconds.  
평균 추론 시간: 0.00003 seconds.  

### Model 4

In [ ]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

model = tf.keras.Sequential([
    DefaultConv2D(64,kernel_size = 5, input_shape=[28, 28, 1]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"),
    DefaultConv2D(64,kernel_size = 5), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"),
])

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(47, activation="softmax"))

Correct Predictions: 16729  
Total Predictions: 18799  
정확도: 0.8899  
훈련시간 : 422.59 seconds.  
평균 추론 시간: 0.0001 seconds.  

## Optimiztion of proposed model

### 기본 모델 구성

In [ ]:
model = tf.keras.Sequential([
    Conv2D(16, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, input_shape=(28, 28, 1), padding='same'),
    AveragePooling2D(),
    Conv2D(32, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(64, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(128, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(128, activation=DENSE_LAYER_ACTIVATION),
    Dropout(0.5),
    Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)
])

In [ ]:
model.compile(optimizer=optimizer,
            loss=LOSS_FUNCTION,
            metrics=METRICS)

callbacks_balance = [
    ModelCheckpoint(CHECK_POINT_BALANCE, verbose=1, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=PATIENCE, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=DECAY_FACTOR, min_lr=MIN_LEARNING_RATE)
]

start_time = time.time()
history_balance = model.fit(train_x_balance, train_y_balance, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_balance, val_y_balance), callbacks=callbacks_balance)
training_time = time.time() - start_time

In [ ]:
start_time = time.time()  
predictions = model.predict(test_x_balance)
end_time = time.time()  
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.4f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.4f} seconds.")

### Learning Rate 0.0005로 변경

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0005
PATIENCE = 10
MIN_LEARNING_RATE = 0.0001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'elu'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = Adam(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

Correct Predictions: 16672  
Total Predictions: 18799  
정확도: 0.8869  
훈련시간 : 215.81 seconds  
평균 추론 시간: 0.0000 seconds  

### MIN_LEARNING_RATE 변경

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0005
PATIENCE = 10
MIN_LEARNING_RATE = 0.00001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'elu'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = Adam(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

Correct Predictions: 16546  
Total Predictions: 18799  
정확도: 0.8802  
훈련시간 : 372.93 seconds  
평균 추론 시간: 0.0001 seconds  

### Batch Size 64로 변경

In [ ]:
BATCH_SIZE = 64
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0005
PATIENCE = 10
MIN_LEARNING_RATE = 0.0001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'elu'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = Adam(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

Correct Predictions: 16344  
Total Predictions: 18799  
정확도: 0.8694  
훈련시간 : 401.15 seconds  
평균 추론 시간: 0.0000 seconds  

### MaxPooling으로 변경

In [ ]:
model = tf.keras.Sequential([
    Conv2D(16, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, input_shape=(28, 28, 1), padding='same'),
    MaxPooling2D(),
    # Conv2D(16, kernel_size=(3, 3), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    # BatchNormalization(),
    # AveragePooling2D(),
    Conv2D(32, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Conv2D(64, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Conv2D(128, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(128, activation=DENSE_LAYER_ACTIVATION),
    Dropout(0.5),
    Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)
])

Correct Predictions: 16596  
Total Predictions: 18799  
정확도: 0.8828  
훈련시간 : 583.83 seconds  
평균 추론 시간: 0.0000 seconds  

### SGD로 변경

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0005
PATIENCE = 10
MIN_LEARNING_RATE = 0.0001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'elu'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = SGD(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

Correct Predictions: 16605  
Total Predictions: 18799  
정확도: 0.8833  
훈련시간 : 672.70 seconds  
평균 추론 시간: 0.0000 seconds  

### Data augmentation 사용

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0001
PATIENCE = 10
MIN_LEARNING_RATE = 0.00001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'elu'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = Adam(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

In [ ]:
history_balance = model.fit(x_combined_shff, y_combined_shff, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(x_combined_val_shff, y_combined_val_shff), callbacks=callbacks_balance)

Correct Predictions: 16845  
Total Predictions: 18799  
정확도: 0.8961  
훈련시간 : 914.54 seconds  
평균 추론 시간: 0.0000 seconds  

### Tanh로 변경

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0001
PATIENCE = 10
MIN_LEARNING_RATE = 0.00001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'tanh'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = Adam(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

Correct Predictions: 16587  
Total Predictions: 18799  
정확도: 0.8823  
훈련시간 : 286.88 seconds  
평균 추론 시간: 0.0000 seconds  

### Relu로 변경[Data augmentation X]

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0001
PATIENCE = 10
MIN_LEARNING_RATE = 0.00001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'relu'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = Adam(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

Correct Predictions: 16612  
Total Predictions: 18799  
정확도: 0.8837  
훈련시간 : 813.88 seconds  
평균 추론 시간: 0.0000 seconds  

### Relu로 변경[Data augmentation]

Correct Predictions: 16832  
Total Predictions: 18799  
정확도: 0.8954  
훈련시간 : 813.88 seconds.  
평균 추론 시간: 0.0000 seconds  

## Final model

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
VALIDATION_SPLIT = 0.15
LEARNING_RATE = 0.0001
PATIENCE = 10
MIN_LEARNING_RATE = 0.00001
DECAY_FACTOR = 0.2
DENSE_LAYER_ACTIVATION = 'elu'
FINAL_LAYER_ACTIVATION = 'softmax'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
optimizer = Adam(learning_rate=LEARNING_RATE)
CHECK_POINT_BALANCE = 'Best_points.h'

In [ ]:
model = tf.keras.Sequential([
    Conv2D(16, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, input_shape=(28, 28, 1), padding='same'),
    AveragePooling2D(),
    # Conv2D(16, kernel_size=(3, 3), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    # BatchNormalization(),
    # AveragePooling2D(),
    Conv2D(32, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(64, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(128, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(128, activation=DENSE_LAYER_ACTIVATION),
    Dropout(0.5),
    Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)
])

In [ ]:
model.compile(optimizer=optimizer,
            loss=LOSS_FUNCTION,
            metrics=METRICS)

callbacks_balance = [
    ModelCheckpoint(CHECK_POINT_BALANCE, verbose=1, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=PATIENCE, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=DECAY_FACTOR, min_lr=MIN_LEARNING_RATE)
]

In [ ]:
start_time = time.time() 
history_balance = model.fit(train_x_balance, train_y_balance, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                                validation_data=(val_x_balance, val_y_balance), callbacks=callbacks_balance)
training_time = time.time() - start_time

start_time = time.time()
predictions = model.predict(test_x_balance)
end_time = time.time() 
total_inference_time = end_time - start_time
average_inference_time = total_inference_time / len(test_x_balance)

predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(test_y_balance, axis=1)

correct_predictions = np.sum(predicted_classes == actual_classes)
accuracy = correct_predictions / len(predicted_classes)

print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {len(predicted_classes)}")
print(f"정확도: {accuracy:.2f}")
print(f"훈련시간 : {training_time:.2f} seconds.")
print(f"평균 추론 시간: {average_inference_time:.5f} seconds.")

Correct Predictions: 16716  
Total Predictions: 18799  
정확도: 0.89  
훈련시간 : 477.71 seconds  
평균 추론 시간: 0.00003 seconds  

## Final model with Data augmentation (Additional process)

#### Data augmentation (same as upper cell)

In [ ]:
# 데이터 스케일링 (StandardScaler 사용)
scaler = StandardScaler()

img_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
)

img_generator2 = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

img_generator3 = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.05,
    height_shift_range=0.05,
)

aug1_train_data = img_generator.flow(train_x_balance,train_y_balance, batch_size=len(train_x_balance))
aug2_train_data = img_generator2.flow(train_x_balance,train_y_balance, batch_size=len(train_x_balance))
aug3_train_data = img_generator3.flow(train_x_balance,train_y_balance, batch_size=len(train_x_balance))

aug1_x,aug1_y = aug1_train_data.next()
aug2_x,aug2_y = aug2_train_data.next()
aug3_x,aug3_y = aug3_train_data.next()

x_combined = np.concatenate((train_x_balance,aug1_x,aug2_x,aug3_x))
y_combined = np.concatenate((train_y_balance,aug1_y,aug2_y,aug3_y))

shuffle_index = np.random.permutation(len(x_combined))
x_combined_shff = x_combined[shuffle_index]
y_combined_shff = y_combined[shuffle_index]


aug1_valid_data = img_generator.flow(val_x_balance,val_y_balance, batch_size=len(val_x_balance))
aug2_valid_data = img_generator2.flow(val_x_balance,val_y_balance, batch_size=len(val_x_balance))
aug3_valid_data = img_generator3.flow(val_x_balance,val_y_balance, batch_size=len(val_x_balance))

aug1_val_x,aug1_val_y = aug1_valid_data.next()
aug2_val_x,aug2_val_y = aug2_valid_data.next()
aug3_val_x,aug3_val_y = aug3_valid_data.next()

x_val_combined = np.concatenate((val_x_balance,aug1_val_x,aug2_val_x,aug3_val_x))
y_val_combined = np.concatenate((val_y_balance,aug1_val_y,aug2_val_y,aug3_val_y))

shuffle_index2 = np.random.permutation(len(x_val_combined))
x_combined_val_shff = x_val_combined[shuffle_index2]
y_combined_val_shff = y_val_combined[shuffle_index2]

#이미지를 2D 배열로 변환
n_samples, height, width, n_channels = x_combined_shff.shape
x_combined_shff = x_combined_shff.reshape((n_samples, -1))

# 학습 데이터 스케일러 학습 및 변환
scaler.fit(x_combined_shff)
x_combined_shff = scaler.transform(x_combined_shff)

# 스케일된 데이터를 다시 원래 이미지 형태로 변환
x_combined_shff = x_combined_shff.reshape((n_samples, height, width, n_channels))

# 검증 데이터 동일한 스케일러로 변환
n_val_samples = x_combined_val_shff.shape[0]
x_combined_val_shff = x_combined_val_shff.reshape((n_val_samples, -1))
x_combined_val_shff = scaler.transform(x_combined_val_shff)
x_combined_val_shff = x_combined_val_shff.reshape((n_val_samples, height, width, n_channels))

# 테스트 데이터 동일한 스케일로 변환
n_val_samples = test_x_balance.shape[0]
test_x_balance = test_x_balance.reshape((n_val_samples, -1))
test_x_balance = scaler.transform(test_x_balance)
test_x_balance = test_x_balance.reshape((n_val_samples, height, width, n_channels))

In [ ]:
model = tf.keras.Sequential([
    Conv2D(16, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, input_shape=(28, 28, 1), padding='same'),
    AveragePooling2D(),
    # Conv2D(16, kernel_size=(3, 3), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    # BatchNormalization(),
    # AveragePooling2D(),
    Conv2D(32, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(64, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(),
    Conv2D(128, kernel_size=(5, 5), activation=DENSE_LAYER_ACTIVATION, padding='same'),
    BatchNormalization(),
    AveragePooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(128, activation=DENSE_LAYER_ACTIVATION),
    Dropout(0.5),
    Dense(number_of_classes, activation=FINAL_LAYER_ACTIVATION)
])

Correct Predictions: 16731   
Total Predictions: 18799  
정확도: 0.89  
훈련시간 : 2399.52 seconds.  
평균 추론 시간: 0.0001 seconds.  